# CSMCC16 Flight Coursework

### Import packages

In [41]:
import numpy as np
import pandas as pd
import datetime
import re

### Import data

In [42]:
#import csv - no error was used when building initial solution
#pass_data = pd.read_csv('AComp_Passenger_data_no_error.csv',header=None)   #comment out as necessary
pass_data = pd.read_csv('AComp_Passenger_data.csv',header=None)   #comment out as necessary
pass_datet = pd.read_csv('AComp_Passenger_data_no_error_DateTime.csv',header=None)
airport_lat_lon = pd.read_csv('Top30_airports_LatLong.csv',header=None)

In [43]:
#check data
#pass_data

In [44]:
# Create as dataframes
pass_data = pd.DataFrame(pass_data)
pass_datetime = pd.DataFrame(pass_datet)
airport_lat_lon = pd.DataFrame(airport_lat_lon)

In [45]:
# Add Column Names
pass_data.columns = ['Passenger_id'
                        ,'Flight_id'
                        ,'From_airport_IATA'
                        ,'Dest_airport_IATA'
                        ,'Depart_time_epoch'
                        ,'Flight_time_min']
airport_lat_lon.columns = ['Airport_name'
                        ,'Airport_IATA_code'
                        ,'Latitude'
                        ,'Longitude']
pass_datetime.columns = ['Passenger_id'
                        ,'Flight_id'
                        ,'From_airport_IATA'
                        ,'Dest_airport_IATA'
                        ,'Departure_date_time'
                        ,'Depart_time_epoch'
                        ,'Flight_time_min'
                        ,'Arrive_date_time'
]

### Remove missing values

In [46]:
#Remove values where passenger id and flight id are missing
pass_data=pass_data.dropna(subset=['Flight_id','Passenger_id'])

### Additional Columns and formats

In [47]:
#Calculate epoch Arrival Time
pass_data['Arrive_epoch'] = pass_data['Depart_time_epoch']+pass_data['Flight_time_min']*60
#Convert epoch to datetime
pass_data['Depart_time_datetime'] = pd.to_datetime(pass_data['Depart_time_epoch'],unit='s')
pass_data['Arrive_time_datetime'] = pd.to_datetime(pass_data['Arrive_epoch'],unit='s')

In [48]:
#And new datetime formats
pass_data['Depart_day_month']=pass_data['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
#pass_data_df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
pass_data['Depart_time']=pass_data['Depart_time_datetime'].dt.strftime("%H:%M:%S")
pass_data['Arrive_time']=pass_data['Arrive_time_datetime'].dt.strftime("%H:%M:%S")

In [49]:
#Add column to allow counting of passengers
pass_data['count']='1'

In [50]:
# Check data
print(pass_data)
print(pass_datetime)
print(airport_lat_lon)

    Passenger_id Flight_id From_airport_IATA Dest_airport_IATA  \
0     UES9151GS5  SQU6245R               DEN               FRA   
1     UES9151GS5  XXQ4064B               JFK               FRA   
2     EZC9678QI6  SOH3431A               ORD               MIA   
3     ONL0812DH1  SOH3431A               ORD               MIA   
4     CYJ0225CH1  PME8178S               DEN               PEK   
..           ...       ...               ...               ...   
504   BWI0520BG6  BER7172M               KUL               LAS   
505   LLZ3798PE3  EWH6301Y               CAN               DFW   
506   KKP5277HZ7  KJR6646J               IAH               BKK   
507   JJM4724RF7  XXQ4064B               JFK               FRA   
508   SJD8775RZ4  WSK1289Z               CLT               vEN   

     Depart_time_epoch  Flight_time_min  Arrive_epoch Depart_time_datetime  \
0           1420564460             1049    1420627400  2015-01-06 17:14:20   
1           1420563917              802    14206120

### Flight data only

In [51]:
#select subset of data 
flight_only_data = pass_data[['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'         
                      ,'Flight_time_min'
                      ,'count'
                        ]]

In [52]:
#Group data by flight
flight_only_data = flight_only_data.groupby(['Flight_id'
                          ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'         
                      ,'Flight_time_min'           
                         ]).count()
#reset index
flight_only_data.reset_index(inplace=True)

#checks
flight_only_data
len(flight_only_data)


54

### Error Correction

In [53]:
#check data
flight_only_data

,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_day_month,Depart_time,Flight_time_min,count
0,4QU6245R,DEN,FRA,Tue 06/Jan/2015,17:14:20,1049,1
1,ATT7791R,A;S,DEN,Tue 06/Jan/2015,17:13:14,1001,1
2,ATT7791R,AM%,DEN,Tue 06/Jan/2015,17:13:14,1001,1
3,ATT7791R,AMS,DEN,Tue 06/Jan/2015,17:13:14,1001,13
4,BER7172M,KUL,LAS,Tue 06/Jan/2015,17:26:07,1848,16
5,BER7172M,KUL,[AS,Tue 06/Jan/2015,17:26:07,1848,1
6,DAU2617A,CGK,SFO,Tue 06/Jan/2015,17:23:06,1811,12
7,DKZ3042O,MIA,SFO,Tue 06/Jan/2015,17:05:27,538,11
8,EWH6301Y,CAN,DFW,Tue 06/Jan/2015,17:22:47,1683,9
9,EWH6301Y,~AN,DFW,Tue 06/Jan/2015,17:22:47,1683,1


In [54]:
#correct formats
format_IATA = r"[A-Z][A-Z][A-Z]"
format_flight_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z]"

#format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
#flight_only_data['Pass_id_format'] =flight_only_data['Passenger_id'].str.contains(format_pass_id)

#add column indicating if correct
flight_only_data['From_airport_format'] =flight_only_data['From_airport_IATA'].str.contains(format_IATA)
flight_only_data['Dest_airport_format'] =flight_only_data['Dest_airport_IATA'].str.contains(format_IATA)
flight_only_data['Flight_id_format'] =flight_only_data['Flight_id'].str.contains(format_flight_id)

In [55]:
#add column indicating if all flight regex correct
flight_only_data['Flight_info_correct']=flight_only_data.all(axis='columns')

In [56]:
#select only those whose regex is correct
flight_info_true = flight_only_data.loc[flight_only_data['Flight_info_correct'] == True]

In [57]:
#Take True airport values, change column names to enable merge

true_airports_from = pd.DataFrame(airport_lat_lon.Airport_IATA_code.unique())
#rename columns to allow merge
true_airports_from.columns = ['From_airport_IATA']
#add column to know if exists
true_airports_from['From_airport_exists']=True

true_airports_dest = pd.DataFrame(airport_lat_lon.Airport_IATA_code.unique())
#rename columns to allow merge
true_airports_dest.columns = ['Dest_airport_IATA']
#add column to know if exists
true_airports_dest['Dest_airport_exists']=True

In [58]:
#send to cv for checking
#merge_with_dest.to_csv(r'merge_with_dest.csv')

In [59]:
#merge with from
merge_with_from=flight_info_true.merge(true_airports_from
                     , how='outer', on=['From_airport_IATA'])
#merge with dest
merge_with_dest=merge_with_from.merge(true_airports_dest
                     , how='outer', on=['Dest_airport_IATA'])
#check data
#merge_with_dest

In [60]:
#drop NaNs - any that are not in the true airport list
merge_with_dest=merge_with_dest.dropna(subset=['From_airport_exists', 'Dest_airport_exists'])

In [63]:
merge_with_dest

,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_day_month,Depart_time,Flight_time_min,count,From_airport_format,Dest_airport_format,Flight_id_format,Flight_info_correct,From_airport_exists,Dest_airport_exists
0,ATT7791R,AMS,DEN,Tue 06/Jan/2015,17:13:14,1001.0,13.0,True,True,True,True,True,True
1,WSK1289Z,CLT,DEN,Tue 06/Jan/2015,16:59:02,278.0,20.0,True,True,True,True,True,True
2,BER7172M,KUL,LAS,Tue 06/Jan/2015,17:26:07,1848.0,16.0,True,True,True,True,True,True
3,QHU1140O,CDG,LAS,Tue 06/Jan/2015,17:14:58,1133.0,20.0,True,True,True,True,True,True
4,VDC9164W,FCO,LAS,Tue 06/Jan/2015,17:18:18,1276.0,14.0,True,True,True,True,True,True
5,MBA8071P,KUL,PEK,Tue 06/Jan/2015,17:04:16,572.0,15.0,True,True,True,True,True,True
6,GMO5938W,LHR,PEK,Tue 06/Jan/2015,17:11:57,1057.0,24.0,True,True,True,True,True,True
7,PME8178S,DEN,PEK,Tue 06/Jan/2015,17:13:29,1322.0,17.0,True,True,True,True,True,True
8,PNE8178S,DEN,PEK,Tue 06/Jan/2015,17:13:29,1322.0,1.0,True,True,True,True,True,True
9,WPW9201U,DFW,PEK,Tue 06/Jan/2015,17:21:09,1452.0,10.0,True,True,True,True,True,True


In [62]:
#select subset of data 
flight_only_data = merge_with_dest[['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time'
                      ,'count'
                        ]]

In [65]:
#Group data to see total flights
flight_only_data = flight_only_data.groupby(['Flight_id'
                        ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time'
                         ]).sum()
flight_only_data

,,,,count
Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time,
ATT7791R,AMS,DEN,17:13:14,13.0
BER7172M,KUL,LAS,17:26:07,16.0
DAU2617A,CGK,SFO,17:23:06,12.0
DKZ3042O,MIA,SFO,17:05:27,11.0
EWH6301Y,CAN,DFW,17:22:47,9.0
FYL5866L,ATL,HKG,17:25:40,20.0
GMO5938W,LHR,PEK,17:11:57,24.0
HUR0974O,DEN,PVG,17:15:25,7.0
HZT2506M,IAH,AMS,17:12:04,13.0


In [66]:
flight_only_data.reset_index(inplace=True)

In [79]:
flight_only_data['Route'] = flight_only_data['From_airport_IATA']+'-' + flight_only_data['Dest_airport_IATA']+'-' + flight_only_data['Depart_time']

In [80]:
flight_only_data=flight_only_data.set_index('Route')
flight_only_data.sort_index()

,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time,count
Route,,,,,
AMS-DEN-17:13:14,ATT7791R,AMS,DEN,17:13:14,13.0
ATL-HKG-17:25:40,FYL5866L,ATL,HKG,17:25:40,20.0
ATL-LHR-17:07:18,XOY7948U,ATL,LHR,17:07:18,16.0
BKK-MIA-17:28:50,YZO4444S,BKK,MIA,17:28:50,16.0
CAN-DFW-17:22:47,EWH6301Y,CAN,DFW,17:22:47,9.0
CAN-DFW-17:23:03,ULZ8130D,CAN,DFW,17:23:03,27.0
CDG-LAS-17:14:58,QHU1140O,CDG,LAS,17:14:58,20.0
CGK-DXB-17:05:58,TMV7633W,CGK,DXB,17:05:58,14.0
CGK-SFO-17:23:06,DAU2617A,CGK,SFO,17:23:06,12.0


In [81]:
flight_only_data.reset_index(inplace=True)

In [82]:
flight_only_data['Route'].duplicated()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14     True
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
Name: Route, dtype: bool

In [ ]:
airport_only_grp = flight_only_true_data.groupby(['From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time_epoch'
                      ,'Flight_time_min'           
                         ]).count()
airport_only_grp

In [ ]:
airport_only_grp.reset_index(inplace=True)

In [ ]:
#data for merging
airport_only_from = airport_only_grp[['From_airport_IATA']]
airport_only_dest = airport_only_grp[['Dest_airport_IATA']]
#rename columns to allow merge
airport_only_from.columns = ['Airport_IATA_code']
airport_only_dest.columns = ['Airport_IATA_code']

In [ ]:
airport_only_from.merge(airport_lat_lon, how='outer', on=['Airport_IATA_code','Airport_IATA_code'])

In [ ]:
#flight info
#flight_from = 
flight_only_true_data[['Flight_id','From_airport_IATA']]
#flight_dest = obj2_results[['Flight_id','Dest_airport_IATA']]

In [ ]:
#rename columns to allow merge
flight_only_grp.columns = ['Flight_id'
                        ,'Airport_IATA_code']
#flight_dest.columns = ['Flight_id'
 #                       ,'Airport_IATA_code']

In [ ]:
flight_from.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])


In [ ]:
#export data
pass_data_df.to_csv(r'check format.csv')

In [ ]:
pass_data_df

In [ ]:
#select data for objective 2
#obj2_data = 
flight_only_data = pass_data_df[['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time_epoch'
                      ,'Flight_time_min'
                        ,'From_airport_format'  
                        ,'Dest_airport_format'
#                        ,'Pass_id_format'
                        ,'Flight_id_format'
                    ,'count'
                        ]]

In [ ]:
#obj2_results = 
flight_only_data.groupby(['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time_epoch'
                      ,'Flight_time_min'
                        ,'From_airport_format'  
                        ,'Dest_airport_format'
                        ,'Pass_id_format'
                        ,'Flight_id_format'                         
                         ]).count()
#reset index as columns
#flight_only_data.reset_index(inplace=True)
##obj2_results

## Objective 1: 
### Determine the number of flights from each airport; include a list of any airports not used.

In [ ]:
# List of top 30 airports
obj_1_list = airport_df['Airport_IATA_code']
obj_1_list.columns = ['From_airport_IATA']
obj_1_list_df = pd.DataFrame(obj_1_list)

In [ ]:
#check data
obj_1_list_df

In [ ]:
# Count number of departure flights
from_df = pass_data_df['From_airport_IATA']
from_counts = from_df.value_counts()
from_counts_df = pd.DataFrame(from_counts)
# move index to column
from_counts_df['Airport_IATA_code'] =from_counts_df.index

In [ ]:
#Check data
print(from_df)
print(from_counts)
print(from_counts_df)

In [ ]:
# merge full list of airports with number of flights from each airport
obj1 = obj_1_list_df.merge(from_counts_df, how='outer', on=['Airport_IATA_code', 'Airport_IATA_code'])

In [ ]:
# clean up results
obj1=obj1.fillna(0)
obj1=obj1.sort_values(by=['From_airport_IATA','Airport_IATA_code'], ascending=[False,True])
obj1.set_index('Airport_IATA_code', inplace=True, drop=True)
obj1

## Objective 2
### Create a list of flights based on the Flight id, this output should include the passenger Id, relevant IATA/FAA codes, the departure time, the arrival time (times to be converted to HH:MM:SS format), and the flight times.

In [ ]:
#select data for objective 2
obj2_data = pass_data_df[['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'
                      ,'Arrive_time'
                      ,'Flight_time_min'
                      ,'Passenger_id']]

## Objective 3
### Calculate the number of passengers on each flight.

In [ ]:
obj2_results = obj2_data.groupby(['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'
                      ,'Arrive_time'
                      ,'Flight_time_min']).count()
#reset index as columns
obj2_results.reset_index(inplace=True)
obj2_results

## Objective 4
### Calculate the line-of-sight (nautical) miles for each flight and the total travelled by each passenger and thus output the passenger having earned the highest air miles.

In [ ]:
#flight info
flight_from = obj2_results[['Flight_id','From_airport_IATA']]
flight_dest = obj2_results[['Flight_id','Dest_airport_IATA']]

In [ ]:
#rename columns to allow merge
flight_from.columns = ['Flight_id'
                        ,'Airport_IATA_code']
flight_dest.columns = ['Flight_id'
                        ,'Airport_IATA_code']

In [ ]:
#merge from flight with latitude and longitude
from_lat_long = flight_from.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])
from_lat_long.columns = ['Flight_id'
                        ,'From_Airport_IATA_code'
                        ,'From_Airport_Name'
                        ,'From_Lat'
                        ,'From_Lon'
                    ]

In [ ]:
#merge dest flight with latitude and longitude
dest_lat_lon = flight_dest.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])
dest_lat_lon.columns = ['Flight_id'
                        ,'Dest_Airport_IATA_code'
                        ,'Dest_Airport_Name'
                        ,'Dest_Lat'
                        ,'Dest_Lon'
                    ]

In [ ]:
#merge from flight with latitude and longitude
from_dest_lat_lon = from_lat_long.merge(dest_lat_lon, how='outer', on=['Flight_id','Flight_id'])

In [ ]:
from_dest_lat_lon

In [ ]:
# Calculate nautical miles (a^2+b^2=c^2)
from_dest_lat_lon['Miles'] = np.sqrt((from_dest_lat_lon['Dest_Lat']-from_dest_lat_lon['From_Lat'])**2
+(from_dest_lat_lon['Dest_Lon']-from_dest_lat_lon['From_Lon'])**2)

In [ ]:
#export results
from_dest_lat_lon.to_csv(r'from_dest_lat_lon.csv')

In [ ]:
# Number of miles per flight 
miles_flight = from_dest_lat_lon[['Flight_id','Miles']]
miles_flight

In [ ]:
#Add flight miles to passenger data
obj4_data = obj2_data.merge(miles_flight, how='outer', on=['Flight_id', 'Flight_id'])
obj4_data

In [ ]:
#Number of miles per flight per passenger
pass_flights= pd.pivot_table(obj4_data, values='miles', index=['Passenger_id'],
               columns=['Flight_id']
              , aggfunc=np.sum
              )
pass_flights

In [ ]:
pass_flights.to_csv(r'pass_flights.csv')

In [ ]:
#Add up flight miles per passenger
Pass_miles = pass_flights.sum(axis = 1, skipna = True) 
Pass_miles_df = pd.DataFrame(Pass_miles)
Pass_miles_df.columns = ['Air_miles']
Pass_miles_df = Pass_miles_df.sort_values(by=['Air_miles'], ascending=False)
Pass_miles_df.to_csv(r'Pass_miles_df.csv')
Pass_miles_df

# old code, maybe useful

In [ ]:
pass_data_df.value_counts(['From_airport_IATA'])

In [ ]:
pass_data_df.groupby(['From_airport_IATA']).count()

In [ ]:
flight_only_grp.isin(flight_only_grp['From_airport_IATA'])

In [ ]:
#pass_data_df.set_index(["From_airport_IATA"]).count(level="From_airport_IATA")
pass_data_df.count(axis="From_airport_IATA")


In [ ]:
#pd.concat([obj_1_list_df, from_counts_df], keys=['Airport_IATA_code','Airport_IATA_code'])


In [ ]:
#pass_data_df['Depart_time_epoch'].dt.strftime("%m/%d/%y")
#pass_data_df['Depart_time_epoch']=
(pd.to_datetime(pass_data_df['Depart_time_epoch'],unit='s')) 


In [ ]:
obj1 = obj_1_list_df.merge(from_counts_df, how='outer', on=['Airport_IATA_code', 'Airport_IATA_code'])

In [ ]:
if re.match(r"hello[0-9]+", 'hello1'):
    print('Yes')

In [ ]:
pattern = r"[A-Z][A-Z][A-Z]"

pd.Series(
     ["DEN", "A;S", "AM%", "3b", "03c", "4dx"],
 ).str.contains(pattern)

In [ ]:
#pass_data_df[pass_data_df['From_airport_IATA'].str.match('DEN')== True]
#pass_data_df['From_airport_format']=
pass_data_df['From_airport_IATA'].str.match('[A-Z][A-Z][A-Z]')

In [ ]:
#merging by IATA code
#select remaining from airports
from_values = flight_info_true.From_airport_IATA.unique()
dest_values = flight_info_true.Dest_airport_IATA.unique()
#turn to dataframe
from_values_df = pd.DataFrame(from_values)
dest_values_df = pd.DataFrame(dest_values)
#rename columns to allow merge
from_values_df.columns = ['Airport_IATA_code']
dest_values_df.columns = ['Airport_IATA_code']

In [ ]:
from_merge['From_airport_exists']=from_merge['Airport_name'].str.contains(r"[A-Z]")
dest_merge['Dest_airport_exists']=dest_merge['Airport_name'].str.contains(r"[A-Z]")
#from_merge[from_merge['Airport_name'].str.contains('NaN')==False]
#new_df = df[df["col"].str.contains(word) == False]